<a href="https://colab.research.google.com/github/fboldt/aulasml/blob/master/aula9b_titanic.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!unzip titanic.zip

Archive:  titanic.zip
  inflating: gender_submission.csv   
  inflating: test.csv                
  inflating: train.csv               


In [ ]:
import pandas as pd
df = pd.read_csv('train.csv')
df.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [ ]:
df.dtypes

,0
PassengerId,int64
Survived,int64
Pclass,int64
Name,object
Sex,object
Age,float64
SibSp,int64
Parch,int64
Ticket,object
Fare,float64


In [ ]:
y = df.Survived
X = df.drop(['Survived'], axis=1)

In [ ]:
for col in X.columns:
    print(f"{col:>12} {len(set(X[col])):4} {X[col].dtype}")

 PassengerId  891 int64
      Pclass    3 int64
        Name  891 object
         Sex    2 object
         Age  265 float64
       SibSp    7 int64
       Parch    7 int64
      Ticket  681 object
        Fare  248 float64
       Cabin  148 object
    Embarked    4 object


In [ ]:
caracteristicas_indesejadas = ['PassengerId', 'Name', 'Ticket', 'Cabin']
Xdrop = X.drop(caracteristicas_indesejadas, axis=1)
Xdrop.columns

Index(['Pclass', 'Sex', 'Age', 'SibSp', 'Parch', 'Fare', 'Embarked'], dtype='object')

In [ ]:
Xnum = Xdrop.select_dtypes('number')
Xnum.columns

Index(['Pclass', 'Age', 'SibSp', 'Parch', 'Fare'], dtype='object')

In [ ]:
for col in Xnum.columns:
    print(f"{col:>12} {Xnum[col].isnull().sum():2}")

      Pclass  0
         Age 177
       SibSp  0
       Parch  0
        Fare  0


In [ ]:
Xnum[-5:]

,Pclass,Age,SibSp,Parch,Fare
886,2,27.0,0,0,13.00
887,1,19.0,0,0,30.00
888,3,NaN,1,2,23.45
889,1,26.0,0,0,30.00
890,3,32.0,0,0,7.75


In [ ]:
from sklearn.impute import SimpleImputer
imputer = SimpleImputer(strategy='median')
XnumTratado = imputer.fit_transform(Xnum)
XnumTratado[-5:]

array([[ 2.  , 27.  ,  0.  ,  0.  , 13.  ],
       [ 1.  , 19.  ,  0.  ,  0.  , 30.  ],
       [ 3.  , 28.  ,  1.  ,  2.  , 23.45],
       [ 1.  , 26.  ,  0.  ,  0.  , 30.  ],
       [ 3.  , 32.  ,  0.  ,  0.  ,  7.75]])

In [ ]:
Xcat = Xdrop.select_dtypes('object')
Xcat.columns

Index(['Sex', 'Embarked'], dtype='object')

In [ ]:
for col in Xcat.columns:
    print(f"{col:>12} {Xcat[col].isnull().sum():2}")

         Sex  0
    Embarked  2


In [ ]:
from sklearn.impute import SimpleImputer
imputer = SimpleImputer(strategy='most_frequent')
XcatTratado = imputer.fit_transform(Xcat)
XcatTratado

array([['male', 'S'],
       ['female', 'C'],
       ['female', 'S'],
       ...,
       ['female', 'S'],
       ['male', 'C'],
       ['male', 'Q']], dtype=object)

In [ ]:
from sklearn.preprocessing import OneHotEncoder
encoder = OneHotEncoder()
XcatTratado = encoder.fit_transform(XcatTratado)
XcatTratado

<891x5 sparse matrix of type '<class 'numpy.float64'>'
	with 1782 stored elements in Compressed Sparse Row format>

In [ ]:
import numpy as np
Xtratado = np.c_[XnumTratado, XcatTratado.toarray()]
Xtratado.shape

(891, 10)

In [ ]:
import pandas as pd
train = pd.read_csv('train.csv')
test = pd.read_csv('test.csv')
train.columns.isin(test.columns)

array([ True, False,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True])

In [ ]:
train.columns[~train.columns.isin(test.columns)]

Index(['Survived'], dtype='object')

In [ ]:
X = train.drop('Survived', axis=1)
y = train.Survived

In [ ]:
from sklearn.base import BaseEstimator, TransformerMixin

class AtributosDesejados(BaseEstimator, TransformerMixin):
    def fit(self, X, y=None):
        self.colunasIndesejadas_ = ['PassengerId', 'Name', 'Ticket', 'Cabin']
        return self
    def transform(self, X, y=None):
        return X.drop(self.colunasIndesejadas_, axis=1)

atributosDesejados = AtributosDesejados()
Xdrop = atributosDesejados.fit_transform(X)
Xdrop.head()

,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked
0,3,male,22.0,1,0,7.2500,S
1,1,female,38.0,1,0,71.2833,C
2,3,female,26.0,0,0,7.9250,S
3,1,female,35.0,1,0,53.1000,S
4,3,male,35.0,0,0,8.0500,S


In [ ]:
class AtributosNumericos(BaseEstimator, TransformerMixin):
    def fit(self, X, y=None):
        self.colunasNumericas_ = X.select_dtypes(include='number').columns
        return self
    def transform(self, X, y=None):
        return X[self.colunasNumericas_]

atributosNumericos = AtributosNumericos()
Xnum = atributosNumericos.fit_transform(Xdrop)
Xnum.columns

Index(['Pclass', 'Age', 'SibSp', 'Parch', 'Fare'], dtype='object')

In [ ]:
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler

pipenum = Pipeline([
    ('atributosNumericos', AtributosNumericos()),
    ('imputer', SimpleImputer(strategy='median')),
    ('scaler', StandardScaler())
])

XnumTratado = pipenum.fit_transform(Xdrop)
XnumTratado

array([[ 0.82737724, -0.56573646,  0.43279337, -0.47367361, -0.50244517],
       [-1.56610693,  0.66386103,  0.43279337, -0.47367361,  0.78684529],
       [ 0.82737724, -0.25833709, -0.4745452 , -0.47367361, -0.48885426],
       ...,
       [ 0.82737724, -0.1046374 ,  0.43279337,  2.00893337, -0.17626324],
       [-1.56610693, -0.25833709, -0.4745452 , -0.47367361, -0.04438104],
       [ 0.82737724,  0.20276197, -0.4745452 , -0.47367361, -0.49237783]])

In [ ]:
class AtributosCategoricos(BaseEstimator, TransformerMixin):
    def fit(self, X, y=None):
        self.colunasCategoricas_ = X.select_dtypes(include='object').columns
        return self
    def transform(self, X, y=None):
        return X[self.colunasCategoricas_]

atributosCategoricos = AtributosCategoricos()
Xcat = atributosCategoricos.fit_transform(Xdrop)
Xcat.columns

Index(['Sex', 'Embarked'], dtype='object')

In [ ]:
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder

pipecat = Pipeline([
    ('atributosCategoricos', AtributosCategoricos()),
    ('imputer', SimpleImputer(strategy='most_frequent')),
    ('encoder', OneHotEncoder())
])

XcatTratado = pipecat.fit_transform(Xdrop)
XcatTratado

<891x5 sparse matrix of type '<class 'numpy.float64'>'
	with 1782 stored elements in Compressed Sparse Row format>

In [ ]:
from sklearn.pipeline import FeatureUnion

unecaracteristicas = FeatureUnion([
    ('pipenum', pipenum),
    ('pipecat', pipecat)
])

Xtratado = unecaracteristicas.fit_transform(Xdrop)
Xtratado

<891x10 sparse matrix of type '<class 'numpy.float64'>'
	with 6237 stored elements in Compressed Sparse Row format>

In [ ]:
preproc = Pipeline([
    ('atributosDesejados', AtributosDesejados()),
    ('unecaracteristicas', unecaracteristicas)
])

Xtratado = preproc.fit_transform(X)
Xtratado

<891x10 sparse matrix of type '<class 'numpy.float64'>'
	with 6237 stored elements in Compressed Sparse Row format>

In [ ]:
from os import pipe
from sklearn.ensemble import RandomForestClassifier

pipeclf = Pipeline([
    ('preproc', preproc),
    ('classificador', RandomForestClassifier())
])

pipeclf.fit(X, y)

Pipeline(steps=[('preproc',
                 Pipeline(steps=[('atributosDesejados', AtributosDesejados()),
                                 ('unecaracteristicas',
                                  FeatureUnion(transformer_list=[('pipenum',
                                                                  Pipeline(steps=[('atributosNumericos',
                                                                                   AtributosNumericos()),
                                                                                  ('imputer',
                                                                                   SimpleImputer(strategy='median')),
                                                                                  ('scaler',
                                                                                   StandardScaler())])),
                                                                 ('pipecat',
                                                                  Pipeline(steps=[('atributosCategoricos',
                                                                                   AtributosCategoricos()),
                                                                                  ('imputer',
                                                                                   SimpleImputer(strategy='most_frequent')),
                                                                                  ('encoder',
                                                                                   OneHotEncoder())]))]))])),
                ('classificador', RandomForestClassifier())])

In [ ]:
from sklearn.metrics import accuracy_score

ypred = pipeclf.predict(X)
accuracy_score(y, ypred)

0.9797979797979798

In [ ]:
from sklearn.model_selection import cross_validate

scores = cross_validate(pipeclf, X, y)
scores, scores['test_score']

({'fit_time': array([1.10028386, 0.93811631, 0.61781621, 0.56636167, 0.59606147]),
  'score_time': array([0.05343914, 0.04861426, 0.03303742, 0.03176475, 0.01534462]),
  'test_score': array([0.78212291, 0.80898876, 0.83707865, 0.76404494, 0.82022472])},
 array([0.78212291, 0.80898876, 0.83707865, 0.76404494, 0.82022472]))

In [ ]:
scores['test_score'].mean()

0.8024919967359236

In [ ]:
pipeclf.fit(X, y)
ypred = pipeclf.predict(test)
ypred

array([0, 0, 0, 1, 1, 0, 0, 0, 1, 0, 0, 0, 1, 0, 1, 1, 0, 1, 0, 1, 1, 1,
       1, 1, 1, 0, 1, 1, 1, 0, 0, 0, 1, 0, 1, 1, 0, 0, 0, 1, 0, 1, 0, 1,
       1, 0, 0, 0, 1, 1, 1, 0, 1, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 1, 1,
       1, 0, 0, 1, 1, 0, 0, 0, 1, 0, 0, 1, 0, 1, 1, 0, 0, 0, 0, 0, 1, 0,
       1, 1, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0,
       0, 1, 1, 0, 1, 0, 0, 1, 0, 1, 1, 0, 1, 0, 0, 1, 0, 1, 0, 0, 1, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 1, 0, 1, 0, 0, 1,
       0, 0, 1, 0, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1,
       1, 1, 1, 1, 0, 0, 1, 0, 1, 0, 1, 0, 0, 0, 0, 0, 1, 0, 1, 0, 1, 0,
       0, 0, 1, 1, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 1, 0, 1, 0,
       1, 0, 1, 1, 1, 0, 0, 1, 0, 0, 0, 1, 0, 0, 1, 0, 1, 1, 1, 1, 1, 1,
       0, 0, 0, 0, 1, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 1,
       0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 1, 0, 0, 0, 0, 0, 1, 1, 0, 1, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0,

In [ ]:
submission = pd.read_csv('gender_submission.csv')
agree = submission['Survived'] == ypred
agree.value_counts()

,count
Survived,
True,341
False,77


In [ ]:
submission['Survived'] = ypred
submission.to_csv('ranfor_submission.csv', index=False)

In [ ]:
from sklearn.model_selection import GridSearchCV

params = {
    'preproc__unecaracteristicas__pipenum__imputer__strategy': ['mean', 'median'],
    'classificador__n_estimators': [10,20,50,100,200],
}

modelo = GridSearchCV(pipeclf, params, cv=5, n_jobs=-1, scoring='accuracy')

scores = cross_validate(modelo, X, y)
scores, scores['test_score'], scores['test_score'].mean()

({'fit_time': array([12.89200544, 10.85026908, 10.64236498, 11.92638159, 11.70867276]),
  'score_time': array([0.01134562, 0.01499486, 0.02332568, 0.00996017, 0.01422811]),
  'test_score': array([0.80446927, 0.78089888, 0.85955056, 0.79213483, 0.82022472])},
 array([0.80446927, 0.78089888, 0.85955056, 0.79213483, 0.82022472]),
 0.8114556525014123)

In [ ]:
from sklearn.experimental import enable_halving_search_cv
from sklearn.model_selection import HalvingGridSearchCV

params = {
    'classificador__n_estimators': [200, 400, 600],
    'classificador__max_features': ['sqrt', 'log2'],
    'classificador__max_depth': [4, 6, 8],
}

modelo = HalvingGridSearchCV(pipeclf, params, aggressive_elimination=False)
scores = cross_validate(modelo, X, y)
scores, scores['test_score'], scores['test_score'].mean()

/usr/local/lib/python3.10/dist-packages/numpy/ma/core.py:2820: RuntimeWarning: invalid value encountered in cast
  _data = np.array(data, dtype=dtype, copy=copy,
/usr/local/lib/python3.10/dist-packages/numpy/ma/core.py:2820: RuntimeWarning: invalid value encountered in cast
  _data = np.array(data, dtype=dtype, copy=copy,


({'fit_time': array([137.9532423 , 111.90413904, 107.13063526, 148.45527983,
         101.18514252]),
  'score_time': array([0.01908398, 0.04496098, 0.04013968, 0.04105759, 0.04516244]),
  'test_score': array([0.82122905, 0.82022472, 0.8258427 , 0.79775281, 0.84831461])},
 array([0.82122905, 0.82022472, 0.8258427 , 0.79775281, 0.84831461]),
 0.8226727763480006)

In [ ]:
modelo.fit(X, y)
modelo.best_params_

{'classificador__max_depth': 4,
 'classificador__max_features': 'log2',
 'classificador__n_estimators': 400}

In [ ]:
submission = pd.read_csv('gender_submission.csv')
ypred = modelo.predict(test)
submission['Survived'] = ypred
submission.to_csv('rftuned_submission.csv', index=False)

In [ ]:
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder

class DenseMatrix(BaseEstimator, TransformerMixin):
    def fit(self, X, y=None):
        return self
    def transform(self, X, y=None):
        return X.toarray()

pipecat = Pipeline([
    ('atributosCategoricos', AtributosCategoricos()),
    ('imputer', SimpleImputer(strategy='most_frequent')),
    ('encoder', OneHotEncoder()),
    ('toarray', DenseMatrix())
])

unecaracteristicas = FeatureUnion([
    ('pipenum', pipenum),
    ('pipecat', pipecat)
])

preproc = Pipeline([
    ('atributosDesejados', AtributosDesejados()),
    ('unecaracteristicas', unecaracteristicas)
])

preproc.fit_transform(test)

array([[ 0.87348191,  0.38623105, -0.49947002, ...,  0.        ,
         1.        ,  0.        ],
       [ 0.87348191,  1.37137004,  0.61699237, ...,  0.        ,
         0.        ,  1.        ],
       [-0.31581919,  2.55353683, -0.49947002, ...,  0.        ,
         1.        ,  0.        ],
       ...,
       [ 0.87348191,  0.70147553, -0.49947002, ...,  0.        ,
         0.        ,  1.        ],
       [ 0.87348191, -0.20485235, -0.49947002, ...,  0.        ,
         0.        ,  1.        ],
       [ 0.87348191, -0.20485235,  0.61699237, ...,  1.        ,
         0.        ,  0.        ]])

In [ ]:
from sklearn.ensemble import VotingClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.linear_model import Perceptron
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression
from sklearn.linear_model import SGDClassifier
from sklearn.neural_network import MLPClassifier

clf = VotingClassifier([
    ('knn', KNeighborsClassifier()),
    ('gnb', GaussianNB()),
    ('per', Perceptron()),
    ('rf', RandomForestClassifier()),
    ('svc', SVC()),
    ('log', LogisticRegression()),
    ('sgd', SGDClassifier()),
    ('mlp', MLPClassifier())
  ])
modelo = Pipeline([
    ('preproc', preproc),
    ('clf', clf)
])
scores = cross_validate(modelo, X, y)
scores, scores['test_score'], scores['test_score'].mean()

/usr/local/lib/python3.10/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:690: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:690: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:690: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:690: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/neural_network/_multilayer_perceptro

({'fit_time': array([2.56305099, 1.21058774, 1.29667258, 2.56562757, 3.9612639 ]),
  'score_time': array([0.1196332 , 0.0535357 , 0.06876731, 0.1538794 , 0.11177802]),
  'test_score': array([0.80446927, 0.82022472, 0.80898876, 0.80337079, 0.85393258])},
 array([0.80446927, 0.82022472, 0.80898876, 0.80337079, 0.85393258]),
 0.8181972255351202)

In [ ]:
submission = pd.read_csv('gender_submission.csv')
modelo.fit(X, y)
ypred = modelo.predict(test)
submission['Survived'] = ypred
submission.to_csv('voting_submission.csv', index=False)

/usr/local/lib/python3.10/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:690: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(


In [ ]:
modelo = Pipeline([
    ('preproc', preproc),
    ('clf', MLPClassifier())
])
scores = cross_validate(modelo, X, y)
scores, scores['test_score'], scores['test_score'].mean()

/usr/local/lib/python3.10/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:690: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:690: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:690: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:690: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/neural_network/_multilayer_perceptro

({'fit_time': array([2.66398907, 2.13498998, 1.00724387, 0.93500805, 0.88593674]),
  'score_time': array([0.02507472, 0.02554536, 0.01041842, 0.01167178, 0.01192784]),
  'test_score': array([0.79329609, 0.80898876, 0.84269663, 0.78651685, 0.87640449])},
 array([0.79329609, 0.80898876, 0.84269663, 0.78651685, 0.87640449]),
 0.8215805661917018)

In [ ]:
submission = pd.read_csv('gender_submission.csv')
modelo.fit(X, y)
ypred = modelo.predict(test)
submission['Survived'] = ypred
submission.to_csv('mlp_submission.csv', index=False)

/usr/local/lib/python3.10/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:690: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(
